Need to evaluate deg = 3, no_features = 2,3,4,5,6.

Executed through /home/b/b309170/scripts/run_era5_evalute_and_transfer_learn_6.sh

In [1]:
# With native Python3 environment -- It takes around 12 minutes to evaluate one day
import os
import sys
import json
import numpy as np
import xarray as xr
import sympy as sp
import matplotlib.pyplot as plt

import scipy as sci
from scipy.optimize import minimize

sys.path.insert(0, '/home/b/b309170/workspace_icon-ml/cloud_cover_parameterization/')
import my_classes
from my_classes import load_data

sys.path.insert(0, '/home/b/b309170/workspace_icon-ml/symbolic_regression/')
from functions import append_dict_to_json

subset_exp = int(sys.argv[1])
number_horizontal_locations = 10**subset_exp

tl_bool = True
SEED = int(sys.argv[2])

num_hours = int(sys.argv[3])

**Load data**

In [ ]:
order_of_vars = ['q', 'clwc', 'ciwc', 't', 'pa', 'zg', 'cc']
data_dict = load_data(source='era5', days='all', order_of_vars=order_of_vars)

TIMESTEPS, VLAYERS, HFIELDS = data_dict['q'].shape

# Removing four upper-most levels
for key in data_dict.keys():
    data_dict[key] = data_dict[key][:, 4:].copy()

# Data output
data_output = data_dict['cc']
del data_dict['cc']

for key in data_dict.keys():
    print(data_dict[key].shape)
    assert data_dict[key].shape == data_dict[key].shape

q
clwc
ciwc
t
pa
cc


In [ ]:
# Add rh
T0 = 273.15
r = 0.00263*data_dict['pa']*data_dict['q']*np.exp((17.67*(data_dict['t']-T0))/(data_dict['t']-29.65))**(-1)
data_dict['rh'] = r

**Reshaping and keeping only the relevant features**

In [ ]:
feat_era = ['rh', 't', 'clwc', 'ciwc']
feat_dya = ['rh', 'ta', 'clw', 'cli']

In [ ]:
# Update loc
loc_era = {}
for i in range(len(feat_era)):
    loc_era[feat_era[i]] = i
    
# Update loc
loc_dya = {}
for i in range(len(feat_dya)):
    loc_dya[feat_dya[i]] = i

In [ ]:
# Keeping only the relevant features
for key in feat_era:
    data_dict[key] = np.reshape(data_dict[key], -1)
    
data_output = np.reshape(data_output, -1)

del data_dict['q']
del data_dict['pa']
del data_dict['zg']

no_features = len(data_dict.keys())

**Cast dict into ndarray**

In [ ]:
# data_array = np.zeros((data_dict['q'].size, len(data_dict.keys())), dtype=np.float32)

k = 0
data_array_not_T = []
for key in feat_era:
    print(key)
    data_array_not_T.append(data_dict[key])
    del data_dict[key]
    k += 1

# Convert into np array and transpose
data_array = np.transpose(np.array(data_array_not_T, dtype=np.float32))

del data_array_not_T

**Pick the subset**

In [ ]:
seed = np.random.seed(SEED)
subset = np.random.randint(0, HFIELDS, number_horizontal_locations)
# Convert to regular int to make check_sum JSON serializable
check_sum = int(np.sum(subset))

# Collecting all grid cell indices for the horizontal fields given by subset
Z = np.zeros((TIMESTEPS, 27, HFIELDS), dtype=int)
for k in range(HFIELDS):
    Z[:,:,k] = k
Z_res = np.reshape(Z, -1)
subset_inds = np.concatenate([np.where(Z_res == s)[0] for s in subset])

In [10]:
train_input = data_array[subset_inds[:num_hours*27]] #num_hours*27
train_output = data_output[subset_inds[:num_hours*27]] #num_hours*27

**Already remove the regime with clw + cli = 0**

In [11]:
reg_0 = np.where(data_array[:, loc_era['clwc']] + data_array[:, loc_era['ciwc']] <= 1e-20)[0]
reg_not_0 = np.where(data_array[:, loc_era['clwc']] + data_array[:, loc_era['ciwc']] > 1e-20)[0]

# Relevant values to compute final MSE/R2-scores
mse_reg_0 = np.mean(data_output[reg_0]**2)
len_reg_0 = len(reg_0)
len_reg_not_0 = len(reg_not_0)
len_data_output = len(data_output)
var_data_output = np.var(data_output)

data_array = data_array[reg_not_0].copy()
data_output = data_output[reg_not_0].copy()

In [12]:
print(mse_reg_0)
print(data_array.shape)
print(data_output.shape)

4.1247884e-05
(19717305, 4)
(19717305,)


**Normalize the features**

In [13]:
all_possible_features = ['hus', 'clw', 'cli', 'ta', 'pa', 'zg', 'fr_land', 'U', 'rh', 'ps', 'hus_z', 'hus_zz', 'clw_z', 'clw_zz', 'cli_z',\
            'cli_zz', 'ta_z', 'ta_zz', 'pa_z', 'pa_zz', 'U_z', 'U_zz', 'rh_z', 'rh_zz']
loc_all = {}
for i in range(len(all_possible_features)):
    loc_all[all_possible_features[i]] = i

# Scale the data
mean_all = [4.12205844e-03,2.25493498e-05,3.38180032e-06,2.57065512e+02,6.00030443e+04,5.64080139e+03,2.35046400e-01,1.32776682e+01,6.02512234e-01,9.86270417e+04,-1.27545273e-06,-4.02484958e-10,1.65204582e-08,-4.34660202e-11,4.29441131e-10,-1.82817316e-12,-4.68742483e-03,-7.54899040e-07,-7.51544542e+00,-1.06989723e-04,1.65615172e-03,-9.27604679e-06,-4.76200071e-05,-1.32246548e-07]
std_all = [5.07648249e-03,5.69702638e-05,1.01308124e-05,3.00533874e+01,3.12514292e+04,5.66963918e+03,4.11184302e-01,1.11389888e+01,3.32494615e-01,6.24039256e+03,2.03179260e-06,1.17041141e-08,1.33311867e-07,1.42840744e-09,6.73384546e-09,5.07424672e-11,5.82875686e-03,6.34826092e-05,3.53136052e+00,1.13215264e-02,6.62892130e-03,6.08144307e-05,2.58065098e-04,2.49552692e-06]

mean = np.concatenate([np.expand_dims(mean_all[loc_all[sel_var]], axis=0) for sel_var in feat_dya], axis = 0)
std = np.concatenate([np.expand_dims(std_all[loc_all[sel_var]], axis=0) for sel_var in feat_dya], axis = 0)

mean = np.float32(mean)
std = np.float32(std)

# Work with scaled training folds
data_scaled = (data_array - mean)/std
train_input = (train_input - mean)/std

del data_array

In [14]:
# Files: polynomial_fit_cl_area_with_derivatives_degree_<deg>_dt_basis_False_no_of_regimes_<no_reg>_regime_<reg>.json
path = '/home/b/b309170/workspace_icon-ml/symbolic_regression/baselines/polynomial_results_v2/dyamond_data/normalized_data/data_driven_regimes/'
    
def polynomial_predictions(n_var,input_data,output_data,coefs):
    assert len(coefs) == n_var + 1
    
    ind = 0
    out_pred = coefs[ind]
    with open(path + 'polynomial_fit_cl_area_with_derivatives_degree_3_dt_basis_False_no_of_regimes_2_regime_1.json', 'r') as file:
        file_content = json.load(file)
    for key in file_content['Number of variables %d'%n_var].keys():
        if not ('Bias' in key or 'R2' in key or 'MSE' in key):
            # Split the square and cube into factors separated by a white space
            key_mod = key
            if '^2' in key:
                key_mod = key.split('^2')[0] + ' ' + key.split('^2')[0].split(' ')[-1] + key.split('^2')[1]
            if '^3' in key:
                key_mod = key.split('^3')[0] + ' ' + key.split('^3')[0].split(' ')[-1] + ' ' + key.split('^3')[0].split(' ')[-1] + key.split('^3')[1]

            # Multiply input features for keys such as 'cli ps'
            prod = 1
            for v in key_mod.split(' '):
                prod *= input_data[:, loc_dya[v]]

            # Add coefficient times feature value to the prediction
            ind += 1
            out_pred += coefs[ind]*prod
            
    out_pred = np.maximum(np.minimum(out_pred, 100), 0)
    
    return out_pred

In [15]:
def objective_polynomials(coefs, X,Y,n_var):
    '''
        The objective function.
    '''
    out_preds = np.minimum(np.maximum(polynomial_predictions(n_var, X,Y,coefs), 0), 100)
    mse = np.mean((out_preds - Y)**2, dtype=np.float64)
    return mse

# Initialize coefficients
coefs = {}
with open(path + 'polynomial_fit_cl_area_with_derivatives_degree_3_dt_basis_False_no_of_regimes_2_regime_1.json', 'r') as file:
    file_content = json.load(file)
    for n_var in [2,3,4,5,6]:
        coefs[n_var] = []
        coefs[n_var].append(file_content['Number of variables %d'%n_var]['Bias'])
        for key in file_content['Number of variables %d'%n_var].keys():
            if not ('Bias' in key or 'R2' in key or 'MSE' in key):
                coefs[n_var].append(file_content['Number of variables %d'%n_var][key])

# Optimize objective
coefs_out = {}
mses_reg_1 = []
for n_var in [2,3,4,5,6]:
    res = minimize(objective_polynomials, coefs[n_var], args=(train_input, train_output, n_var), \
                   method='BFGS', options={'disp': True})
    coefs_out['deg_3_n_var_%d'%n_var] = res.x
    
    # MSE on the entire dataset
    if tl_bool:
        preds = polynomial_predictions(n_var,data_scaled,data_output,res.x)
    else:
        preds = polynomial_predictions(n_var,data_scaled,data_output,coefs[n_var])
    mses_reg_1.append(np.mean((preds - data_output)**2, dtype=np.float64))

Optimization terminated successfully.
         Current function value: 80.503913
         Iterations: 26
         Function evaluations: 144
         Gradient evaluations: 36
Optimization terminated successfully.
         Current function value: 35.186235
         Iterations: 29
         Function evaluations: 165
         Gradient evaluations: 33
Optimization terminated successfully.
         Current function value: 31.057822
         Iterations: 45
         Function evaluations: 294
         Gradient evaluations: 49
Optimization terminated successfully.
         Current function value: 21.454107
         Iterations: 39
         Function evaluations: 385
         Gradient evaluations: 55
Optimization terminated successfully.
         Current function value: 20.852094
         Iterations: 60
         Function evaluations: 600
         Gradient evaluations: 75


In [16]:
mses_total = [(mse_reg_0*len_reg_0 + mse_reg_1*len_reg_not_0)/len_data_output for mse_reg_1 in mses_reg_1]

results = {}
for n in range(2, 7):
    parent_key = 'deg_3_n_feat_%d_tl_%d_num_hours_%d_seed_%d'%(n,subset_exp,num_hours,SEED)
    results[parent_key] = {}
    results[parent_key]['MSE'] = mses_total[n-2]
    results[parent_key]['R2'] = 1 - mses_total[n-2]/var_data_output
    results[parent_key]['Coefs'] = list(coefs_out['deg_3_n_var_%d'%n_var])
    results[parent_key]['Check sum'] = check_sum

**Save results**

In [17]:
# Dump results
append_dict_to_json(results, '/home/b/b309170/workspace_icon-ml/symbolic_regression/evaluate_schemes/on_era5/results/era5_1979-2021/sel_polyn_deg_3_BFGS.json')